Starting with importing all packages needed for this notebook. 

In [8]:
import os
import shap
import math
import sys
print(sys.version)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import tensorflow as tf
print(tf.__version__)
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras import regularizers
import pickle

    
np.random.seed(42)


3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
2.15.0


Reading in the name of all environmental variables and creating a new file which will display all the metrics of the best performing model for each species. 

In [9]:
file_dir=('/Users/maddie/Projects/fishpredict')
#create text file to store results in and close again:
with open(file_dir+'/results/DNN_performance/DNN_eval.txt','w+') as file:
    file.write("Species"+"\t"+"Test_loss"+"\t"+"Test_acc"+"\t"+"Test_tpr"+"\t"+"Test_AUC"+"\t"+"Test_LCI95%"+"\t"+"Test_UCI95%"+"\t"+"occ_samples"+"\t"+"abs_samples"+"\n")
    file.close()
#access file with list of taxa names
taxa=pd.read_csv(file_dir+'/data/modified_data/gbif_filtered/taxa_list.txt',header=None)
taxa.columns=["taxon"] 

###column variable names
with open(file_dir+'/data/data_raw/bio_oracle/variable_list.txt') as f:
      new_cols = f.readlines()


Constructing and testing the model for each species. 

* For every species that I had, I constructed and trained the model 5 times, and saved the model with the best metrics for each species. 
  
* During each of the 5 runs per species, there were 40 epochs per model, using the ADAM optimizer and a learning rate of 0.001. 
  
* All species data was split 85/15 for training and 'evaluating' (valiation, but it is shown as testing here). 
  
* The model with the best AUC and other metrics was saved in an .h5 model which was used for both testing in step 5 and for predicitons in the web app. 

In [11]:
var_names=[]

for item in new_cols:
    item=item.replace("\n","")
    var_names.append(item) 

    
for species in taxa["taxon"][:]:
   
    #open dataframe and rename columns
    spec = species
    table = pd.read_csv(file_dir +"/data/modified_data/spec_ppa_env/%s_env_dataframe.csv"%spec)         
    table.rename(columns=dict(zip(table.columns[1:10], var_names)),inplace=True)
    
    ####################################
    #  filter dataframe for training   #
    ####################################
   
    # drop any row with no-data values
    table = table.dropna(axis=0, how="any")


    # make feature vector
    band_columns = [column for column in table.columns[1:10]]
    

    X = []
    y = []

    for _, row in table.iterrows():
        x = row[band_columns].values
        x = x.tolist()
        x.append(row["present/pseudo_absent"])
        X.append(x)

    df = pd.DataFrame(data=X, columns=band_columns + ["presence"])
    df.to_csv("filtered.csv", index=None)

    # extract n. of occ. and abs. samples
    occ_len=int(len(df[df["presence" ]==1]))
    abs_len=int(len(df[df["presence" ]==0]))
    
    ####################################
    #  Numpy feature and target array  #
    ####################################

    X = []
    y = []

    band_columns = [column for column in df.columns[:-1]]

    for _, row in df.iterrows():
        X.append(row[band_columns].values.tolist())
        y.append([1 - row["presence"], row["presence"]])

    X = np.vstack(X)
    y = np.vstack(y)


    

    ####################################
    #    Split training and test set   #
    ####################################

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y,random_state=42)
    
    test_set=pd.DataFrame(X_test)
    test_set.rename(columns=dict(zip(test_set.columns[0:9], var_names)),inplace=True)
    
    shuffled_X_train=X_train.copy()
    np.random.shuffle(shuffled_X_train)
    shuffled_X_train=shuffled_X_train[:1000] # random subsample from test set for feature importance
    
    shuffled_X_test=X_test.copy()
    np.random.shuffle(shuffled_X_test)
    shuffled_X_test=shuffled_X_test[:1000] # random subsample from test set for feature importance
    
    ####################################
    #      Training and testing        #
    ####################################
    
    # prepare metrics
    test_loss=[]
    test_acc=[]
    test_AUC=[]
    test_tpr=[]
    test_uci=[]
    test_lci=[]

   
    Best_model_AUC=[0]
    
    # Five repetitions
    for i in range(1,6):
        print("run %s"%i)
        ###################
        # Construct model #
        ###################
        batch_size = 75
        num_classes = 2
        epochs = 40

        num_inputs = X.shape[1]  # number of features


        model = Sequential()
        layer_1 = Dense(100, activation='relu',input_shape=(num_inputs,))#, kernel_regularizer=regularizers.l1(0.000001))
        layer_2 = Dense(50, activation='relu', input_shape=(num_inputs,))#, kernel_regularizer=regularizers.l1(0.000001))
        layer_3 = Dense(25, activation='relu', input_shape=(num_inputs,))#, kernel_regularizer=regularizers.l1(0.0000001))
        layer_4 = Dense(10, activation='relu', input_shape=(num_inputs,))#, kernel_regularizer=regularizers.l1(0.00000001))


        model.add(layer_1)
        model.add(Dropout(0.3))
        model.add(layer_2)
        model.add(Dropout(0.5))
        model.add(layer_3)
        model.add(Dropout(0.3))
        model.add(layer_4)
        model.add(Dropout(0.5))


        out_layer = Dense(num_classes, activation=None)
        model.add(out_layer)
        model.add(Activation("softmax"))

        #model.summary()

        model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics =['accuracy'])
        
        ###############
        # Train model #
        ###############
        
        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
        
    


        #history = model.fit(X_train, y_train, epochs=epochs, batch_size = batch_size, verbose=0)
        #shap_values = shap.Explainer(model)(X_test)

        ##############
        # Test model #
        ##############
        score = model.evaluate(X_test, y_test, verbose=1)
        predictions = model.predict(X_test)
        fpr, tpr, thresholds = roc_curve(y_test[:, 1], predictions[:, 1])
        len_tpr=int(len(tpr)/2)
      

        #################
        # Append scores #
        #################
        test_loss.append(score[0])
        test_acc.append(score[1])
        test_AUC.append(roc_auc_score(y_test[:, 1], predictions[:, 1]))
        test_tpr.append(tpr[len_tpr])
        AUC = roc_auc_score(y_test[:, 1], predictions[:, 1])

        ###############################
        # Create confidence intervals #
        ###############################
        n_bootstraps=1000
        y_pred=predictions[:,1]
        y_true=y_test[:,1]
        rng_seed=42
        bootstrapped_scores =[]


        rng=np.random.RandomState(rng_seed)
        for i in range (n_bootstraps):
            #bootstrap by sampling with replacement on prediction indices
            indices = rng.randint(0,len(y_pred)-1,len(y_pred))
            if len (np.unique(y_true[indices])) <2:
                continue

            score = roc_auc_score(y_true[indices],y_pred[indices])
            bootstrapped_scores.append(score)

        sorted_scores=np.array(bootstrapped_scores)
        sorted_scores.sort()

        ci_lower=sorted_scores[int(0.05*len(sorted_scores))]
        ci_upper=sorted_scores[int(0.95*len(sorted_scores))]
     
        test_lci.append(ci_lower)
        test_uci.append(ci_upper)
       
    
        ##############################################################
        # Selection of best model across runs and feature importance #
        ##############################################################
    
        #determine whether new model AUC is higher
        if AUC > Best_model_AUC[0]:
            # if yes save model to disk / overwrite previous model
            Best_model_AUC[0]=AUC
            model_filename = file_dir + '/deploy_webapp/saved_models/{}.pkl'.format(spec)
            os.makedirs(os.path.dirname(model_filename), exist_ok=True)
            with open(model_filename, 'wb') as model_file:
                pickle.dump(model, model_file)

            #with open(file_dir + '/deploy_webapp/saved_models/{}.pkl'.format(spec), 'wb') as model_file:
            #    pickle.dump(model, model_file)
            #model.save(file_dir+'/deploy_webapp/saved_models/{}.pkl'.format(spec))
            #model_json=model.to_json()
            #with open (file_dir+'/results/fish/{}/{}_model.json'.format(spec,spec),'w') as json_file:
            #    json_file.write(model_json)
            #model.save_weights(file_dir+'/results/fish/{}/{}_model.h5'.format(spec,spec))
            #if yes, save a figure of shap feature value impact    
            
            if int(len(X_train)) > 5000:           
                explainer=shap.DeepExplainer(model,shuffled_X_train)
                test_set=pd.DataFrame(shuffled_X_test)
                test_set.rename(columns=dict(zip(test_set.columns[0:40], var_names)),inplace=True)
                
                shap_values=explainer.shap_values(shuffled_X_test)
                fig=shap.summary_plot(shap_values[1],test_set,show=False)
                plt.savefig(file_dir+'/results/fish/{}/{}_feature_impact'.format(spec,spec),bbox_inches="tight")
                plt.close()
            
            else:
                explainer=shap.DeepExplainer(model,X_train)
                shap_values=explainer.shap_values(X_test)
                fig=shap.summary_plot(shap_values[1],test_set,show=False)
                plt.savefig(file_dir+'/results/fish/{}/{}_feature_impact'.format(spec,spec),bbox_inches="tight")
                plt.close()
            


    # Model output metrics averaged across five runs to be written to file
    avg_loss= sum(test_loss)/len(test_loss)
    avg_acc = sum(test_acc)/len(test_acc)
    avg_AUC = sum(test_AUC)/len(test_AUC)
    avg_tpr = sum(test_tpr)/len(test_tpr)
    avg_lci = sum(test_lci)/len(test_lci)
    avg_uci = sum(test_uci)/len(test_uci)

    #Write to file
    with open(file_dir+'/results/DNN_performance/DNN_eval.txt','a') as file:
        file.write(spec+"\t"+str(avg_loss)+"\t"+str(avg_acc)+"\t"+str(avg_tpr)+"\t"+str(avg_AUC)+"\t"+str(avg_lci)+"\t"+str(avg_uci)+"\t"+str(occ_len)+"\t"+str(abs_len)+"\n")       


    #Next species!


run 1
232/232 [==============================] - 0s 935us/step


keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


run 2
232/232 [==============================] - 0s 916us/step


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


run 3
232/232 [==============================] - 0s 839us/step
run 4
232/232 [==============================] - 0s 834us/step
run 5
232/232 [==============================] - 0s 824us/step
run 1
132/132 [==============================] - 0s 956us/step


...

run 3
397/397 [==============================] - 1s 1ms/step


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


run 4
397/397 [==============================] - 1s 1ms/step


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


run 5
397/397 [==============================] - 1s 1ms/step
